In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk #nltk means natural language processing toolkit
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

#using the SQLite Table to read data
con = sqlite3.connect('database.sqlite')

#filtering only positive and negative reviews i.e. not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT * 
FROM Reviews
WHERE Score !=3
""",con
)

# Give reviews withe Score>3 a positive rating, and reviews with a score<3 a negative review
def partition(x):
    if x<3:
        return 'negative'
    return 'positive'
actualScore = filtered_data['Score'] # here 'Score' is a coloumn of sqlite3 and the value which is not equal to 3 which's stores in actual score
positiveNegative = actualScore.map(partition) #using partition method where returns only positive or negative value
filtered_data['Score'] = positiveNegative #and finally positive or negative strings are stored in filtered_data's score column

In [2]:
filtered_data.shape
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
display = pd.read_sql_query("""
SELECT * FROM Reviews WHERE Score !=3 AND UserId="AR5J8UI46CURR"
ORDER By ProductID
""",con
)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [5]:
#sorting data according to ProductId in ascending order
sorted_data = filtered_data.sort_values('ProductId',axis=0,ascending=True)
sorted_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...
...,...,...,...,...,...,...,...,...,...,...
176791,191721,B009UOFTUI,AJVB004EB0MVK,D. Christofferson,0,0,negative,1345852800,weak coffee not good for a premium product and...,"This coffee supposedly is premium, it tastes w..."
1362,1478,B009UOFU20,AJVB004EB0MVK,D. Christofferson,0,0,negative,1345852800,weak coffee not good for a premium product and...,"This coffee supposedly is premium, it tastes w..."
303285,328482,B009UUS05I,ARL20DSHGVM1Y,Jamie,0,0,positive,1331856000,Perfect,The basket was the perfect sympathy gift when ...
5259,5703,B009WSNWC4,AMP7K1O84DH1T,ESTY,0,0,positive,1351209600,DELICIOUS,Purchased this product at a local store in NY ...


In [6]:
#Deduplication of entries
final = sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False) #just keeping the first value and remove the other entries
final.shape

(364173, 10)

In [7]:
#checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100 
#here final data is after removing the duplicate data and filtered data is totall data where 3 does not belongs to

69.25890143662969

In [8]:
display = pd.read_sql_query("""
SELECT * 
FROM Reviews 
WHERE Score !=3 AND Id=44737 OR Id=64422
ORDER By ProductID
""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [9]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
print(final.shape)
#How many positives and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

In [10]:
#BoW
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final['Text'].values) #collects total bag of words

In [11]:
type(final_counts)

scipy.sparse.csr.csr_matrix

In [12]:
final_counts.get_shape()

(364171, 115281)

In [17]:
CountVectorizer??

In [22]:
#find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if(len(re.findall('<.*?>',sent))):
        print(i)
        print(sent)
        break;
    i += 1;  #here there are a sentence with different punctuation mark

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [33]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop = set(stopwords.words('english')) #set of stop words
sno = nltk.stem.SnowballStemmer('english') #initailizing snow ball stemmer
def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>') #the words between the angular brackets and remove with spaces
    cleantext = re.sub(cleanr,' ',sentence) 
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence) #for single cout(') we have to put this \ instead of logical or(|)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return cleaned 
print(stop)
print('******************************')
print(sno.stem('tasty'))

{'o', 'where', "wouldn't", "weren't", 'is', 'same', 'just', 'up', 'more', 'we', 'hers', 'me', 'be', "didn't", 'over', 'been', 'ain', 're', "you'll", 'most', "aren't", "haven't", 'from', 'now', "that'll", 'm', 'an', 'each', 'other', 'our', 'once', 'not', 'so', 've', 'was', 'll', 'should', 'ma', "you're", 'very', "mightn't", 'which', 'having', 'here', 'for', 'mustn', "hasn't", 'too', 'ourselves', 't', 'this', 'into', 'yourselves', 'further', 'on', 'did', 'his', 'the', 'why', "you've", 'all', 's', "you'd", 'again', 'haven', 'ours', 'down', "isn't", 'who', 'these', 'when', 'wasn', 'how', 'her', 'any', 'my', 'they', "shan't", 'it', 'wouldn', 'out', 'y', 'him', 'off', 'that', 'have', 'aren', 'your', 'its', 'won', "won't", 'needn', "needn't", 'their', 'whom', 'some', 'doing', 'own', 'has', "it's", 'weren', 'nor', 'you', 'do', 'until', 'himself', 'while', 'those', 'if', 'by', 'after', 'few', 'a', 'yourself', 'shouldn', "doesn't", 'them', 'only', "should've", 'will', "hadn't", "shouldn't", "cou

In [27]:
import nltk
nltk.download('stopwords') # download the stop words from nltk

[nltk_data] Downloading package stopwords to C:\Users\Ibrahim Al
[nltk_data]     Azhar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [51]:
#code for implementing step by step the checks mentioned in the pre processor
i=0
str1 = ' '
final_string = []
all_positive_words = [] #store words from +ve reviews here
all_negative_words = [] #store words from -ve reviews here
s=''
for sent in final['Text'].values:
    filtered_sentence = []
    sent=cleanhtml(sent) #remove HTML tags
    for w in sent.split(): # split the sentence means create a list where each item is word 
        for cleaned_words in cleanpunc(w).split(): # pass the w in cleanpuc method for removing punctuation
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)): # the word is alpha check this word has only letter
                if(cleaned_words.lower() not in stop):  # the word is not in stop word
                    s=(sno.stem(cleaned_words.lower())).encode('utf8') # apply the snow ball stemmer
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
                        
    #print(filterd_sentences)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    final_string.append(str1)
    i +=1

In [36]:
e=final['Text'][10]

In [37]:
e

"I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!"

In [41]:
p=e.split()

In [45]:
p

['I',
 "don't",
 'know',
 'if',
 "it's",
 'the',
 'cactus',
 'or',
 'the',
 'tequila',
 'or',
 'just',
 'the',
 'unique',
 'combination',
 'of',
 'ingredients,',
 'but',
 'the',
 'flavour',
 'of',
 'this',
 'hot',
 'sauce',
 'makes',
 'it',
 'one',
 'of',
 'a',
 'kind!',
 'We',
 'picked',
 'up',
 'a',
 'bottle',
 'once',
 'on',
 'a',
 'trip',
 'we',
 'were',
 'on',
 'and',
 'brought',
 'it',
 'back',
 'home',
 'with',
 'us',
 'and',
 'were',
 'totally',
 'blown',
 'away!',
 'When',
 'we',
 'realized',
 'that',
 'we',
 'simply',
 "couldn't",
 'find',
 'it',
 'anywhere',
 'in',
 'our',
 'city',
 'we',
 'were',
 'bummed.<br',
 '/><br',
 '/>Now,',
 'because',
 'of',
 'the',
 'magic',
 'of',
 'the',
 'internet,',
 'we',
 'have',
 'a',
 'case',
 'of',
 'the',
 'sauce',
 'and',
 'are',
 'ecstatic',
 'because',
 'of',
 'it.<br',
 '/><br',
 '/>If',
 'you',
 'love',
 'hot',
 'sauce..I',
 'mean',
 'really',
 'love',
 'hot',
 'sauce,',
 'but',
 "don't",
 'want',
 'a',
 'sauce',
 'that',
 'tasteles

In [44]:
q.split()

['know']

In [53]:
final['CleanedText'] = final_string

In [55]:
final.head(3) #below the processed review can be seen in the CleanedText
#store final table into an SQlLite table for future
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews',conn,schema=None,if_exists='replace')


In [56]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
print("Most Common Positive Words : ",freq_dist_positive.most_common(20))
print("Most Common Negative Words : ",freq_dist_negative.most_common(20))

Most Common Positive Words :  [(b'like', 139429), (b'tast', 129047), (b'good', 112766), (b'flavor', 109624), (b'love', 107357), (b'use', 103888), (b'great', 103870), (b'one', 96726), (b'product', 91033), (b'tri', 86791), (b'tea', 83888), (b'coffe', 78814), (b'make', 75107), (b'get', 72125), (b'food', 64802), (b'would', 55568), (b'time', 55264), (b'buy', 54198), (b'realli', 52715), (b'eat', 52004)]
Most Common Negative Words :  [(b'tast', 34585), (b'like', 32330), (b'product', 28218), (b'one', 20569), (b'flavor', 19575), (b'would', 17972), (b'tri', 17753), (b'use', 15302), (b'good', 15041), (b'coffe', 14716), (b'get', 13786), (b'buy', 13752), (b'order', 12871), (b'food', 12754), (b'dont', 11877), (b'tea', 11665), (b'even', 11085), (b'box', 10844), (b'amazon', 10073), (b'make', 9840)]


In [57]:
#bi-gram, tri-gram and n-gram
#removing stop words like "not" should be avoided before building n-grams
count_vect = CountVectorizer(ngram_range=(1,2)) #it takes only uni gram and bi gram
final_bigram_counts = count_vect.fit_transform(final['Text'].values) #takes total text

In [58]:
final_bigram_counts.get_shape()

(364171, 2910192)

In [59]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [63]:
final_tf_idf.get_shape()


(364171, 2910192)

In [64]:
features = tf_idf_vect.get_feature_names()
len(features)

2910192

In [65]:
features[100000:100010]

['ales until',
 'ales ve',
 'ales would',
 'ales you',
 'alessandra',
 'alessandra ambrosia',
 'alessi',
 'alessi added',
 'alessi also',
 'alessi and']

In [66]:
#convert a row in sparse matrix to a numpy array
print(final_tf_idf[3,:].toarray()[0])

[0. 0. 0. ... 0. 0. 0.]


In [67]:
def top_tfidf_feats(row,features,top_n=25):
    topn_ids = np.argsort(row)[::-1][:top_n] #argsort sorting the top 25 features
    top_feats = [(features[i],row[i]) for i in topn_ids] #the loop takes features and row from top 25 and put it into top_feats
    df = pd.DataFrame(top_feats) #insert into a dataframe
    df.columns = ['feature','tfidf'] #add coloumn names 'feature' and 'tfidf'
    return df
top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25) #here final_tf_idf is vector corresponding to review and converts into numpy array which is row parameter
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


In [ ]:
# Using Google News Word2Vectors
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.b')


In [ ]:
model.wv['computer']

In [ ]:
model.wv.similarity('woman','man')

In [ ]:
# Train your own Word2Vec model using your own text corpus
# Converts the Text into a list
#firstly split transferred a sentence to a words and makes one list
#secondly split transferred each words makes a separte list

i=0
list_of_sent = []
for sent in final['Text'].values: #for each sentence of reviews,values takes a full sentence
    filtered_sentence = []
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue
    list_of_sent.append(filtered_sentence)

In [68]:
print(final['Text'].values[0])
print("****************************")
print(list_of_sent[0])

this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
****************************


NameError: name 'list_of_sent' is not defined

In [ ]:
w2v_model = genism.models.Word2Vec(list_of_sent,min_count=5,size=50,workers=4)

In [ ]:
words = list(w2v_model.wv.vocab)
print(len(words))

In [ ]:
w2v_model.wv.most_similar('tasty')

In [ ]:
w2v_model.wv.most_similar('like')

In [ ]:
count_vect_feat = count_vect.get_features_names() # list of words in the BoW
count_vect_feat.index('like') #count_vect is list of uni-grams and bi-grams and here finding the index 'like' among the uni-grams and bi-grams
print(count_vect_feat[64055])

In [ ]:
#avg W2V
sent_vectors = [];
for sent in list_of_sent:
    sent_vec = np.zeros(50)
    cnt_words=0;
    for word in sent: #for each word in sentence
        try:
            vec =w2v_model.wv[word] #applying w2v in each word of a sentence
            sent_vec += vec
            cnt_words +=1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))
            

In [ ]:
# TF-IDF weighted Word2vec
tfidf_feat = tf_idf_vect.get_feature_names()
tfidf_sent_vectors = [];
row=0;
for sent in list_of_sent:
    sent_vec = np.zeros(50)
    weight_sum =0;
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            tfidf = final_tf_idf[row,tfidf_feat.index[word]]
            sent_vec += (vec*tf_idf)
            weight_sum +=tf_idf
        except:
            pass
    sent_vec /=weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row +=1